In [1]:
import xgboost as xgb


ImportError: No module named 'xgboost'

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
# optional
import seaborn as sns; sns.set()
import math
from sklearn.model_selection import GridSearchCV
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import GradientBoostingClassifier
from scipy.sparse.linalg import svds
from functools import partial


train = pd.read_table("../Data_competition/X_train.txt", sep=',',header=None)
test = pd.read_table("../Data_competition/X_test.txt",sep=',',header=None)
cible=pd.read_table("../Data_competition/y_train.txt",sep=',',header=None)
header=pd.read_csv("../Data_competition/header.csv",sep=',',header=None)

feature_name=header.drop(header.columns[[2]], axis=1)
feature_name=feature_name.drop(feature_name.index[[56,49,30,20,14,0,38]])[3]
feature_name
feature_name2=header.drop(header.columns[[2]], axis=1)
feature_name2=feature_name2.drop(feature_name2.index[[56,49,30,20,14,0,38]])[1]
                                 
train_via_def=train
train_via_def.columns=feature_name2
test_via_def=test
test_via_def.columns=feature_name2

frames=[train_via_def,test_via_def]
alldata= pd.concat(frames,keys=['train','test'])



##########################


def emsvd(Y, k=None, tol=1E-3, maxiter=None):
    """
    Approximate SVD on data with missing values via expectation-maximization

    Inputs:
    -----------
    Y:          (nobs, ndim) data matrix, missing values denoted by NaN/Inf
    k:          number of singular values/vectors to find (default: k=ndim)
    tol:        convergence tolerance on change in trace norm
    maxiter:    maximum number of EM steps to perform (default: no limit)

    Returns:
    -----------
    Y_hat:      (nobs, ndim) reconstructed data matrix
    mu_hat:     (ndim,) estimated column means for reconstructed data
    U, s, Vt:   singular values and vectors (see np.linalg.svd and 
                scipy.sparse.linalg.svds for details)
    """

    if k is None:
        svdmethod = partial(np.linalg.svd, full_matrices=False)
    else:
        svdmethod = partial(svds, k=k)
    if maxiter is None:
        maxiter = np.inf

    # initialize the missing values to their respective column means
    mu_hat = np.nanmean(Y, axis=0, keepdims=1)
    valid = np.isfinite(Y)
    Y_hat = np.where(valid, Y, mu_hat)

    halt = False
    ii = 1
    v_prev = 0

    while not halt:

        # SVD on filled-in data
        U, s, Vt = svdmethod(Y_hat - mu_hat)

        # impute missing values
        Y_hat[~valid] = (U.dot(np.diag(s)).dot(Vt) + mu_hat)[~valid]

        # update bias parameter
        mu_hat = Y_hat.mean(axis=0, keepdims=1)

        # test convergence using relative change in trace norm
        v = s.sum()
        if ii >= maxiter or ((v - v_prev) / v_prev) < tol:
            halt = True
        ii += 1
        v_prev = v

    return Y_hat, mu_hat, U, s, Vt
    
    

alldata_fill_svd=pd.DataFrame(emsvd(alldata)[0],columns=feature_name2)
#alldata_fill_svd['x51']=alldata_fill_svd['x51']-2011
                     
###########

#train_nan_svd=alldata_fill_svd.iloc[0:2400]
#test_nan_svd=alldata_fill_svd.iloc[2400:4001]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:80: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


In [5]:
train_fill_svd=pd.DataFrame(emsvd(train_via_def)[0],columns=feature_name2)
train_fill_svd['x51']=train_fill_svd['x51']-2011

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:80: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


In [ ]:
tuned_parameters = [{'loss': ['deviance','exponential'],
                    'max_features':[5,10,20,50],
                    'max_depth':[3,5,10,30],
                    'n_estimators':[100,300]}]

clf = GridSearchCV(GradientBoostingClassifier(), tuned_parameters, cv=10,verbose=1)
clf.fit(train_fill_svd, cible[0].values)


print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()

print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()

In [ ]:
alldata_nan_svd=pd.DataFrame(emsvd(alldata)[0],columns=feature_name2)
alldata_nan_svd['x51']=alldata_nan_svd['x51']-2011

In [ ]:
test_nan_svd_2=alldata_nan_svd.iloc[2400:4001]

In [ ]:
y_pred_clf=clf.predict(test_nan_svd_2)

index=range(len(y_pred_clf))
data=np.transpose([index,y_pred_clf])
a = pd.DataFrame(data=data,columns=['Id','Prediction'])
a.to_csv("submission4.csv", sep=",",index=False)